# Project 14: Image Augmentation

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
import torchvision.transforms as transforms

In [15]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(
        degrees = 7,
        translate = (0, 0.07),
        shear = 7,
        scale = (1, 1.2)
    ),
    transforms.ToTensor()
])

In [16]:
transform_test = transforms.ToTensor()

In [17]:
train = datasets.MNIST(root='Datasets', download=True, train=True, transform=transform_train)
test = datasets.MNIST(root='Datasets', download=True, train=False, transform=transform_test)

In [18]:
train_loader = torch.utils.data.DataLoader(train, batch_size=128)
test_loader = torch.utils.data.DataLoader(test, batch_size=128)

In [19]:
class classifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, (3,3))
        self.activation = nn.ReLU()
        self.pool = nn.MaxPool2d((2,2))
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(32*13*13, 128)
        self.output = nn.Linear(128, 10)

    def forward(self, x):
        X = self.pool(self.activation(self.conv1(x)))
        X = self.flatten(X)
        X = self.activation(self.linear(X))
        X = self.output(X)

        return X

In [20]:
net = classifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(device)


classifier(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (activation): ReLU()
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=5408, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=10, bias=True)
)

In [22]:
def training_loop(loader, epoch):

    running_loss = 0.0
    running_accuracy = 0.0

    for i, data in enumerate(loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        ps = F.softmax(outputs)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy = torch.mean(equals.type(torch.FloatTensor))

        running_accuracy += accuracy.item()

        print(f'Epoch {epoch+1}, Batch {i+1} - Loss: {loss:.3f}, Accuracy: {accuracy:.3f}')

    print(f'Epoch {epoch+1} - Loss: {running_loss/len(loader):.3f}, Accuracy: {running_accuracy/len(loader):.3f}')

In [24]:
for epoch in range(2):
    print('Training...')
    training_loop(train_loader, epoch)

    net.eval()
    print('Testing...')
    training_loop(test_loader, epoch)

Training...
Epoch 1, Batch 1 - Loss: 0.946, Accuracy: 0.688
Epoch 1, Batch 2 - Loss: 0.863, Accuracy: 0.719


/tmp/ipykernel_25671/1296387940.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  ps = F.softmax(outputs)


Epoch 1, Batch 3 - Loss: 0.648, Accuracy: 0.820
Epoch 1, Batch 4 - Loss: 0.798, Accuracy: 0.750
Epoch 1, Batch 5 - Loss: 0.959, Accuracy: 0.656
Epoch 1, Batch 6 - Loss: 0.808, Accuracy: 0.734
Epoch 1, Batch 7 - Loss: 0.893, Accuracy: 0.742
Epoch 1, Batch 8 - Loss: 1.004, Accuracy: 0.695
Epoch 1, Batch 9 - Loss: 0.951, Accuracy: 0.617
Epoch 1, Batch 10 - Loss: 0.820, Accuracy: 0.734
Epoch 1, Batch 11 - Loss: 0.958, Accuracy: 0.672
Epoch 1, Batch 12 - Loss: 0.837, Accuracy: 0.758
Epoch 1, Batch 13 - Loss: 0.768, Accuracy: 0.805
Epoch 1, Batch 14 - Loss: 0.768, Accuracy: 0.789
Epoch 1, Batch 15 - Loss: 0.774, Accuracy: 0.758
Epoch 1, Batch 16 - Loss: 0.698, Accuracy: 0.758
Epoch 1, Batch 17 - Loss: 0.703, Accuracy: 0.789
Epoch 1, Batch 18 - Loss: 0.797, Accuracy: 0.719
Epoch 1, Batch 19 - Loss: 0.654, Accuracy: 0.820
Epoch 1, Batch 20 - Loss: 0.869, Accuracy: 0.734
Epoch 1, Batch 21 - Loss: 0.789, Accuracy: 0.789
Epoch 1, Batch 22 - Loss: 0.754, Accuracy: 0.680
Epoch 1, Batch 23 - Loss: 0